In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib.request
import time
import io
import sys
import asyncio
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_autoinstaller
path = chromedriver_autoinstaller.install()
from parsel import Selector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import winsound as ws

#exec_path = 'D:\\xref\\scrape\\webdriver\\chromedriver'
pd.set_option('display.max_colwidth', None)
driver = webdriver.Chrome(executable_path = path)

import warnings
warnings.filterwarnings("ignore")

# STARTUP_DELAY = 1
# NEXT_PAGE_DELAY =1
# WAIT_AFTER_ELEMENT_LOADED = 1
# DELAY_TO_CHECK_IF_MEW_ELEMENT_IS_VISIBLE =1
# LOAD_SPECIFIC_RESULT_DELAY = 1
# BACK_TO_MAIN_LIST_DELAY = 1

In [2]:
raw_text = ['MD']#, 'Saint Charls County MO']
replace_dict = {"MD": "MARYLAND STATE"}
for key, value in replace_dict.items():
    raw_text = [item.replace(key, value) for item in raw_text]
    
search_string_list = ["Diagnostic Imaging","Radiology", "MRI","CT SCAN"]

In [3]:
print("Raw text:\n", raw_text)

print("search string:\n", search_string_list)

Raw text:
 ['MARYLAND STATE']
search string:
 ['Diagnostic Imaging', 'Radiology', 'MRI', 'CT SCAN']


In [4]:
#Obtain information for Businesses from google map
df_level_1 = pd.DataFrame(columns=["LocName","LocHREF"])

N = 10
idx = 0

for prefix in raw_text:
    t1 = pd.Timestamp.now()
    df_level_1.to_pickle('Imaging_MD_level_1_scrape_intermediate.pkl')
    prefix = prefix.replace(' ',"+")
    print(prefix)
    for search_string in search_string_list:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + prefix
        url_local = 'https://www.google.com/maps/search/' + url_local
        print(url_local)
        driver.get(url_local)
        driver.maximize_window()
        time.sleep(1.5)

        driver.execute_script("document.body.style.zoom='zoom 50%'") #does not work!!

        keep_looping = True
        page_number = 0
        while page_number <= N:
            action = webdriver.ActionChains(driver)
            soup = BeautifulSoup(driver.page_source, 'lxml')
    #         print(soup)
            
            for button_tmp in soup.find_all('button'):
                try:

                    if button_tmp['aria-label'] == ' Next page ':
                        button_id = button_tmp['id']

                except:
                    pass

            try:
                python_button = driver.find_element_by_id(button_id)
                action.move_to_element(python_button)
                action.perform()
                action.move_by_offset(35, -50)
                action.double_click()
                action.perform()
                time.sleep(.25)
                action.double_click()
                action.perform()
                time.sleep(.25)
                action.double_click()
                action.perform()
                time.sleep(.25)
                action.double_click()
                action.perform()
                time.sleep(.25)
                action.double_click()
                action.perform()
                time.sleep(.25)
            except:
                pass
            page_content = driver.page_source
    #         print(page_content)


            response = Selector(page_content)

            for el in response.xpath('//div[contains(@aria-label, "Results for")]/div/div[./a]'):
                LocHREF = el.xpath('./a/@href').extract_first('')
                df_level_1.loc[idx,"LocHREF"] = LocHREF

                LocName= el.xpath('./a/@aria-label').extract_first('')
                df_level_1.loc[idx,"LocName"] = LocName

                Business = el.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]').extract_first('')
                Business = re.search(r'<span jstcache="\d{3}">(.*)<\/span>',Business).group(1)
                df_level_1.loc[idx,"Business"] = Business
                
                df_level_1.loc[idx,"raw text"] = prefix
                df_level_1.loc[idx,"search string"] = search_string  
                
                idx += 1

#             if (page_number+1)%10==0:
#                 df_level_1.to_csv("google_map_result_{}_{}_{}.csv".format(prefix,search_string,int(page_number+1)))
#                 df_level_1 = pd.DataFrame(columns=["LocName","LocHREF"])
#                 idx = 0
        
            page_number += 1
        
            if page_number <= N:
                try:
                    python_button = driver.find_element_by_id(button_id)
                    python_button.click()
                    time.sleep(1)
                except:
                    #print('reached end of good buttons')
                    # keep_looping = False
                    pass

        df_level_1.to_csv("google_map_result_{}_{}.csv".format(prefix,search_string),index = False)#,int(page_number)))              
#         driver.quit();
                
df_level_1

MARYLAND+STATE
Diagnostic+Imaging
https://www.google.com/maps/search/Diagnostic+Imaging+MARYLAND+STATE
Radiology
https://www.google.com/maps/search/Radiology+MARYLAND+STATE
MRI
https://www.google.com/maps/search/MRI+MARYLAND+STATE
CT+SCAN
https://www.google.com/maps/search/CT+SCAN+MARYLAND+STATE


,LocName,LocHREF,Business,raw text,search string
0,University of Maryland Medical Center Diagnostic Radiology,https://www.google.com/maps/place/University+of+Maryland+Medical+Center+Diagnostic+Radiology/data=!4m7!3m6!1s0x89c803589a3a927d:0x7a6cfc042016847d!8m2!3d39.2880132!4d-76.6248449!16s%2Fg%2F12hqyz0dc!19sChIJfZI6mlgDyIkRfYQWIAT8bHo?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
1,Diagnostic Imaging Largo,https://www.google.com/maps/place/Diagnostic+Imaging+Largo/data=!4m7!3m6!1s0x89b7c0086aa9c1f5:0x3366c7286632e556!8m2!3d38.9057617!4d-76.8385848!16s%2Fg%2F1v26m_m6!19sChIJ9cGpagjAt4kRVuUyZijHZjM?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
2,Community Radiology Associates | Olney Imaging Center,https://www.google.com/maps/place/Community+Radiology+Associates+%7C+Olney+Imaging+Center/data=!4m7!3m6!1s0x89b7d11fe15b692d:0xc605a6bdda2b3085!8m2!3d39.1539734!4d-77.0557827!16s%2Fg%2F1tdz0vq3!19sChIJLWlb4R_Rt4kRhTAr2r2mBcY?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
3,Community Radiology Associates | Clinton Imaging Center,https://www.google.com/maps/place/Community+Radiology+Associates+%7C+Clinton+Imaging+Center/data=!4m7!3m6!1s0x89b7bcbdd11c8249:0xc9add3fcb084c331!8m2!3d38.7652068!4d-76.9015944!16s%2Fg%2F1tjmfk15!19sChIJSYIc0b28t4kRMcOEsPzTrck?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
4,Dianostic Imaging Services,https://www.google.com/maps/place/Dianostic+Imaging+Services/data=!4m7!3m6!1s0x89c9ecdfaad8a667:0xd08875a6f116f327!8m2!3d39.6354011!4d-77.6965266!16s%2Fg%2F1tfts1k3!19sChIJZ6bYqt_syYkRJ_MW8aZ1iNA?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
...,...,...,...,...,...
819,Green Imaging,https://www.google.com/maps/place/Green+Imaging/data=!4m7!3m6!1s0x8640c0626579b575:0x131a775f338e188d!8m2!3d29.7048697!4d-95.4264602!16s%2Fg%2F1tfxh1vg!19sChIJdbV5ZWLAQIYRjRiOM193GhM?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,CT+SCAN
820,InSight Medical Imaging,https://www.google.com/maps/place/InSight+Medical+Imaging/data=!4m7!3m6!1s0x880e571f4993d7d7:0xe80887765291631a!8m2!3d41.8078498!4d-88.093193!16s%2Fg%2F11gslsynvn!19sChIJ19eTSR9XDogRGmORUnaHCOg?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,CT+SCAN
821,The University of Texas MD Anderson Cancer Center,https://www.google.com/maps/place/The+University+of+Texas+MD+Anderson+Cancer+Center/data=!4m7!3m6!1s0x8640c0769ce6cd5b:0xcf9deec312b7e8e0!8m2!3d29.7068296!4d-95.3971193!16zL20vMDN6cDh4!19sChIJW83mnHbAQIYR4Oi3EsPunc8?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,CT+SCAN
822,Vanderbilt University Medical Center,https://www.google.com/maps/place/Vanderbilt+University+Medical+Center/data=!4m7!3m6!1s0x8864669885aa512b:0xfa6fe5b7d3c37d5a!8m2!3d36.1425227!4d-86.8014009!16s%2Fm%2F04g3cl3!19sChIJK1GqhZhmZIgRWn3D07flb_o?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,CT+SCAN


In [5]:
# Read CSV files from List
# data = pd.concat(map(pd.read_csv, ['google_map_result_MARYLAND_Diagnostic+Imaging.csv'
#                                    , 'google_map_result_MARYLAND+State_CT+SCAN.csv'
#                                    ,'google_map_result_MARYLAND+State_Diagnostic+Imaging.csv'
#                                    ,'google_map_result_MARYLAND+State_MRI.csv'
#                                    ,'google_map_result_MARYLAND+State_Radiology.csv']))
file_names = ['google_map_result_MARYLAND+STATE_CT+SCAN.csv'
              ,'google_map_result_MARYLAND+STATE_MRI.csv'
              ,'google_map_result_MARYLAND+STATE_Radiology.csv'
              ,'google_map_result_MARYLAND+STATE_Diagnostic+Imaging.csv']

data = pd.concat((pd.read_csv(i) for i in file_names)).reset_index(drop = True) # Import
data.head(5)

,LocName,LocHREF,Business,raw text,search string
0,University of Maryland Medical Center Diagnostic Radiology,https://www.google.com/maps/place/University+of+Maryland+Medical+Center+Diagnostic+Radiology/data=!4m7!3m6!1s0x89c803589a3a927d:0x7a6cfc042016847d!8m2!3d39.2880132!4d-76.6248449!16s%2Fg%2F12hqyz0dc!19sChIJfZI6mlgDyIkRfYQWIAT8bHo?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
1,Diagnostic Imaging Largo,https://www.google.com/maps/place/Diagnostic+Imaging+Largo/data=!4m7!3m6!1s0x89b7c0086aa9c1f5:0x3366c7286632e556!8m2!3d38.9057617!4d-76.8385848!16s%2Fg%2F1v26m_m6!19sChIJ9cGpagjAt4kRVuUyZijHZjM?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
2,Community Radiology Associates | Olney Imaging Center,https://www.google.com/maps/place/Community+Radiology+Associates+%7C+Olney+Imaging+Center/data=!4m7!3m6!1s0x89b7d11fe15b692d:0xc605a6bdda2b3085!8m2!3d39.1539734!4d-77.0557827!16s%2Fg%2F1tdz0vq3!19sChIJLWlb4R_Rt4kRhTAr2r2mBcY?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
3,Community Radiology Associates | Clinton Imaging Center,https://www.google.com/maps/place/Community+Radiology+Associates+%7C+Clinton+Imaging+Center/data=!4m7!3m6!1s0x89b7bcbdd11c8249:0xc9add3fcb084c331!8m2!3d38.7652068!4d-76.9015944!16s%2Fg%2F1tjmfk15!19sChIJSYIc0b28t4kRMcOEsPzTrck?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging
4,Dianostic Imaging Services,https://www.google.com/maps/place/Dianostic+Imaging+Services/data=!4m7!3m6!1s0x89c9ecdfaad8a667:0xd08875a6f116f327!8m2!3d39.6354011!4d-77.6965266!16s%2Fg%2F1tfts1k3!19sChIJZ6bYqt_syYkRJ_MW8aZ1iNA?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging


In [6]:
data.shape

(2087, 5)

In [7]:
data = data.drop_duplicates(keep='first').reset_index()

In [8]:
data.shape
data.to_csv(r'test.csv')
data.shape

(764, 6)

In [9]:
# Obtain business information for each individual location
for i in range(len(data)):
    t = pd.Timestamp.now()
    url = data.loc[i, "LocHREF"]#"https://www.google.com/maps/place/CORA+Physical+Therapy+South+County/data=!4m6!3m5!1s0x87d8cf35f7d3f769:0x7b3019c994319873!8m2!3d38.5039686!4d-90.3788281!16s%2Fg%2F1pp2tvx8w?authuser=0&hl=en&rclk=1"
    driver.get(url)
    time.sleep(1)
    try:
        Name = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
        data.loc[i,"Name"] = Name
    except:
        pass

    try:
        Business_b = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button').text
        data.loc[i,"Business_b"] = Business_b
    except:
        pass

    try:
        Address  = driver.find_element_by_xpath('//button[contains(@ aria-label,"Address")]').text
        data.loc[i,"Address"] = Address
    except:
        pass

    try:
        AddressDetail = driver.find_element_by_xpath('//div[contains(@ aria-label,"Located in")]').text
        data.loc[i,"AddressDetail"] = AddressDetail
    except:
        pass

    try:
        WebSite = driver.find_element_by_xpath('//a[contains(@ aria-label,"Website")]').text
        data.loc[i,"WebSite"] = WebSite
    except:
        pass

    print(i,"\t",t, data.loc[i,"LocName"],Name, Business_b, Address , WebSite)    
    data.to_csv("google_map_result_{}_{}_B.csv".format(prefix,search_string),index = False)
    
data.head(5)

0 	 2022-07-07 16:01:04.774864 University of Maryland Medical Center Diagnostic Radiology University of Maryland Medical Center Diagnostic Radiology Medical diagnostic imaging center 22 South Greene St First floor, Baltimore, MD 21201 umms.org
1 	 2022-07-07 16:01:07.400217 Diagnostic Imaging Largo Diagnostic Imaging Largo Medical diagnostic imaging center 1100 Mercantile Ln Suite 150, Upper Marlboro, MD 20774 facebook.com
2 	 2022-07-07 16:01:09.900447 Community Radiology Associates | Olney Imaging Center Community Radiology Associates | Olney Imaging Center Medical diagnostic imaging center 18111 Prince Philip Dr ste t-20, Olney, MD 20832 radnet.com
3 	 2022-07-07 16:01:12.053579 Community Radiology Associates | Clinton Imaging Center Community Radiology Associates | Clinton Imaging Center Medical diagnostic imaging center 9131 Piscataway Rd Ste. 370 & MRI Ste. 110, Clinton, MD 20735 radnet.com
4 	 2022-07-07 16:01:14.290461 Dianostic Imaging Services Dianostic Imaging Services X-ray

40 	 2022-07-07 16:02:45.008799 Community Radiology Associates l Crestwood Imaging Center Community Radiology Associates l Crestwood Imaging Center Medical diagnostic imaging center 7211 Bank Ct Suite 100 & 101, Frederick, MD 21703 advancedradiology.com
41 	 2022-07-07 16:02:47.852836 Community Radiology Associates | The MRI Center at Greenbelt Community Radiology Associates | The MRI Center at Greenbelt Medical diagnostic imaging center 7525 Greenway Center Dr Ste. T1, Greenbelt, MD 20770 radnet.com
42 	 2022-07-07 16:02:51.424026 Adventist HealthCare Imaging Adventist HealthCare Imaging Medical diagnostic imaging center 9601 Blackwell Rd Suite 150, Rockville, MD 20850 adventisthealthcare.com
43 	 2022-07-07 16:02:54.171204 Community Radiology Associates | Montgomery Community MRI Community Radiology Associates | Montgomery Community MRI MRI center 18103 Prince Philip Dr, Olney, MD 20832 radnet.com
44 	 2022-07-07 16:02:57.161334 Adventist HealthCare Imaging Adventist HealthCare Imagi

82 	 2022-07-07 16:04:43.449233 Progressive Radiology Progressive Radiology MRI center 7701 Greenbelt Rd #102, Greenbelt, MD 20770 progressiveradiology.com
83 	 2022-07-07 16:04:46.455265 Radiology Imaging Associates, Corporate Office Radiology Imaging Associates, Corporate Office Medical diagnostic imaging center 7801 Old Branch Ave, Clinton, MD 20735 riassociates.com
84 	 2022-07-07 16:04:49.268471 Ascension Saint Agnes Imaging Catonsville Ascension Saint Agnes Imaging Catonsville Medical diagnostic imaging center 6501 Baltimore National Pike D, Catonsville, MD 21228 healthcare.ascension.org
85 	 2022-07-07 16:04:51.545931 Advanced Radiology -- White Square Advanced Radiology -- White Square Medical diagnostic imaging center 9101 Franklin Square Dr, Baltimore, MD 21237 advancedradiology.com
86 	 2022-07-07 16:04:54.301023 Advanced Radiology Advanced Radiology Hospital 724 Maiden Choice Ln, Catonsville, MD 21228 advancedradiology.com
87 	 2022-07-07 16:04:56.930920 Ascension Saint Agn

126 	 2022-07-07 16:06:39.780537 University of Maryland Charles Regional Medical Center Inpatient Imaging Services University of Maryland Charles Regional Medical Center Inpatient Imaging Services Medical diagnostic imaging center 5 Garrett Ave, La Plata, MD 20646 umms.org
127 	 2022-07-07 16:06:42.248889 Luminis Health Imaging Annapolis Luminis Health Imaging Annapolis Diagnostic center 2002 Medical Pkwy #100, Annapolis, MD 21401 aahs.org
128 	 2022-07-07 16:06:44.700917 Johns Hopkins Schools of Medical Imaging Johns Hopkins Schools of Medical Imaging Medical school 111 Market Pl #830, Baltimore, MD 21202 somi.jh.edu
129 	 2022-07-07 16:06:47.443231 Chesapeake Medical Imaging Chesapeake Medical Imaging Radiologist 810 Bestgate Rd Suite 100, Annapolis, MD 21401 cmirad.net
130 	 2022-07-07 16:06:49.815151 Medical Imaging of Fredericksburg Medical Imaging of Fredericksburg Medical diagnostic imaging center 1201 Sam Perry Blvd #102, Fredericksburg, VA 22401 mifimaging.com
131 	 2022-07-07

171 	 2022-07-07 16:08:46.153295 Fair Oaks Imaging Center - Reston Radiology Fair Oaks Imaging Center - Reston Radiology Medical diagnostic imaging center 4001 Fair Ridge Dr Suite 103, Fairfax, VA 22033 restonradiology.com
172 	 2022-07-07 16:08:48.670128 Hagerstown Imaging Hagerstown Imaging Medical diagnostic imaging center 11236 Robinwood Dr #106, Hagerstown, MD 21742 hagerstownimaging.org
173 	 2022-07-07 16:08:51.157865 GW Radiology GW Radiology Medical diagnostic imaging center 900 23rd St NW, Washington, DC 20037 gwhospital.com
174 	 2022-07-07 16:08:53.434772 LifeBridge Health Department of Radiology LifeBridge Health Department of Radiology Hospital department 2401 W Belvedere Ave, Baltimore, MD 21215 lifebridgehealth.org
175 	 2022-07-07 16:08:56.512371 Inova Fairfax Hospital Imaging Center Inova Fairfax Hospital Imaging Center Medical diagnostic imaging center 3300 Gallows Rd, Falls Church, VA 22042 inova.org
176 	 2022-07-07 16:08:58.704341 Fairfax Radiology Center of Sprin

...

623 	 2022-07-07 16:27:49.869986 University of Maryland Medical Center Diagnostic Radiology University of Maryland Medical Center Diagnostic Radiology Medical diagnostic imaging center 22 South Greene St First floor, Baltimore, MD 21201 umms.org
624 	 2022-07-07 16:27:52.314185 UM Shore Regional Health Diagnostic and Imaging Center at Easton UM Shore Regional Health Diagnostic and Imaging Center at Easton Medical diagnostic imaging center 10 Martin Ct, Easton, MD 21601 umms.org
625 	 2022-07-07 16:27:54.727746 American Radiology Services | Howard County American Radiology Services | Howard County Medical diagnostic imaging center 8820 Columbia 100 Pkwy Ste 100, Columbia, MD 21045 radnet.com
626 	 2022-07-07 16:27:57.376338 MedStar Health: Southern Maryland Imaging Center MedStar Health: Southern Maryland Imaging Center X-ray lab 7501 Surratts Rd # 104, Clinton, MD 20735 medstarhealth.org
627 	 2022-07-07 16:27:59.895875 Laurel Radiology Services Laurel Radiology Services Medical diagno

666 	 2022-07-07 16:29:37.279060 Advanced Diagnostic Radiology Advanced Diagnostic Radiology Doctor 952 Seton Dr #210, Cumberland, MD 21502 adronline.org
667 	 2022-07-07 16:29:39.435928 Office of Health Care Quality Office of Health Care Quality State government office 7120 Samuel Morse Dr, Second Floor, Columbia, MD 21046 health.maryland.gov
668 	 2022-07-07 16:29:41.431292 State Anatomy Board-Maryland State Anatomy Board-Maryland State government office 655 W Baltimore St, Baltimore, MD 21201 health.maryland.gov
669 	 2022-07-07 16:29:43.919997 Central Booking And Intake Center - RELEASE Central Booking And Intake Center - RELEASE Department of Public Safety E Madison St, Baltimore, MD 21202 news.maryland.gov
670 	 2022-07-07 16:29:46.437414 Maryland Board of Nursing Maryland Board of Nursing License bureau 4140 Patterson Ave, Baltimore, MD 21215 mbon.maryland.gov
671 	 2022-07-07 16:29:48.924160 Ellicott City Pediatric Associates,PA Ellicott City Pediatric Associates,PA Pediatricia

711 	 2022-07-07 16:31:35.041622 Maryland-State Board of Cosmetologists Maryland-State Board of Cosmetologists State government office 500 N Calvert St #201, Baltimore, MD 21202 dllr.state.md.us
712 	 2022-07-07 16:31:38.280016 UPMC Western Maryland Outpatient Diagnostic Center UPMC Western Maryland Outpatient Diagnostic Center Medical office 12400 Willowbrook Rd., First Floor, Cumberland, MD 21502 wmhs.com
713 	 2022-07-07 16:31:40.372045 Maryland State Board Of Elections Maryland State Board Of Elections State government office 151 West St #200, Annapolis, MD 21401 elections.maryland.gov
714 	 2022-07-07 16:31:42.542397 Secretary of State Secretary of State State government office 16 Francis St # 1, Annapolis, MD 21401 sos.state.md.us
715 	 2022-07-07 16:31:45.131639 Office of the Public Defender Office of the Public Defender State government office 6 St Paul St #1400, Baltimore, MD 21202 opd.state.md.us
716 	 2022-07-07 16:31:47.691563 Baltimore Office of the Comptroller Baltimore O

757 	 2022-07-07 16:33:34.581478 SimonMed Imaging - Maryland Parkway SimonMed Imaging - Maryland Parkway Medical diagnostic imaging center 3061 S Maryland Pkwy Ste 102, Las Vegas, NV 89109 simonmed.com
758 	 2022-07-07 16:33:36.940269 Brigham Womens Hospital Radiology Brigham Womens Hospital Radiology Radiologist 75 Francis St, Boston, MA 02115 brighamandwomens.org
759 	 2022-07-07 16:33:39.818070 Green Imaging Green Imaging Medical diagnostic imaging center Corporate Offices, No imaging provided at this site, 3139 W Holcombe Blvd #A104, Houston, TX 77025 greenimaging.net
760 	 2022-07-07 16:33:42.452434 InSight Medical Imaging InSight Medical Imaging Medical diagnostic imaging center 2009 Warrenville Rd, Lisle, IL 60532 insightmrict.com
761 	 2022-07-07 16:33:45.032931 The University of Texas MD Anderson Cancer Center The University of Texas MD Anderson Cancer Center Cancer treatment center 1515 Holcombe Blvd, Houston, TX 77030 mdanderson.org
762 	 2022-07-07 16:33:48.080252 Vanderbil

,index,LocName,LocHREF,Business,raw text,search string,Name,Business_b,Address,WebSite
0,0,University of Maryland Medical Center Diagnostic Radiology,https://www.google.com/maps/place/University+of+Maryland+Medical+Center+Diagnostic+Radiology/data=!4m7!3m6!1s0x89c803589a3a927d:0x7a6cfc042016847d!8m2!3d39.2880132!4d-76.6248449!16s%2Fg%2F12hqyz0dc!19sChIJfZI6mlgDyIkRfYQWIAT8bHo?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging,University of Maryland Medical Center Diagnostic Radiology,Medical diagnostic imaging center,"22 South Greene St First floor, Baltimore, MD 21201",umms.org
1,1,Diagnostic Imaging Largo,https://www.google.com/maps/place/Diagnostic+Imaging+Largo/data=!4m7!3m6!1s0x89b7c0086aa9c1f5:0x3366c7286632e556!8m2!3d38.9057617!4d-76.8385848!16s%2Fg%2F1v26m_m6!19sChIJ9cGpagjAt4kRVuUyZijHZjM?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging,Diagnostic Imaging Largo,Medical diagnostic imaging center,"1100 Mercantile Ln Suite 150, Upper Marlboro, MD 20774",facebook.com
2,2,Community Radiology Associates | Olney Imaging Center,https://www.google.com/maps/place/Community+Radiology+Associates+%7C+Olney+Imaging+Center/data=!4m7!3m6!1s0x89b7d11fe15b692d:0xc605a6bdda2b3085!8m2!3d39.1539734!4d-77.0557827!16s%2Fg%2F1tdz0vq3!19sChIJLWlb4R_Rt4kRhTAr2r2mBcY?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging,Community Radiology Associates | Olney Imaging Center,Medical diagnostic imaging center,"18111 Prince Philip Dr ste t-20, Olney, MD 20832",radnet.com
3,3,Community Radiology Associates | Clinton Imaging Center,https://www.google.com/maps/place/Community+Radiology+Associates+%7C+Clinton+Imaging+Center/data=!4m7!3m6!1s0x89b7bcbdd11c8249:0xc9add3fcb084c331!8m2!3d38.7652068!4d-76.9015944!16s%2Fg%2F1tjmfk15!19sChIJSYIc0b28t4kRMcOEsPzTrck?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging,Community Radiology Associates | Clinton Imaging Center,Medical diagnostic imaging center,"9131 Piscataway Rd Ste. 370 & MRI Ste. 110, Clinton, MD 20735",radnet.com
4,4,Dianostic Imaging Services,https://www.google.com/maps/place/Dianostic+Imaging+Services/data=!4m7!3m6!1s0x89c9ecdfaad8a667:0xd08875a6f116f327!8m2!3d39.6354011!4d-77.6965266!16s%2Fg%2F1tfts1k3!19sChIJZ6bYqt_syYkRJ_MW8aZ1iNA?authuser=0&hl=en&rclk=1,Medical diagnostic imaging center,MARYLAND+STATE,Diagnostic+Imaging,Dianostic Imaging Services,X-ray lab,"1101 Opal Ct #111, Hagerstown, MD 21740",diagnosticimagingservices.com


In [ ]:
data.to_csv()

In [13]:
# Obtain latitude and longitude for each Location
# url = 'https://www.mapdevelopers.com/geocode_tool.php'
# driver.get(url)
for i in range(len(data)):
    t = pd.Timestamp.now()
    url = 'https://www.mapdevelopers.com/geocode_tool.php'
    driver.get(url)

    x = data.loc[i, 'Address']
    try:
        what_search_box = driver.find_element_by_id("address")
        what_search_box.clear()
        what_search_box.send_keys(x)
        what_search_box.send_keys(Keys.ENTER)
        time.sleep(1)    

        data.loc[i,"Lat"] = driver.find_element_by_id('display_lat').text
        data.loc[i,"Long"] = driver.find_element_by_id('display_lng').text
    except:
        pass
    print(i,pd.Timestamp.now() - t, data.loc[i,['Lat','Long']])
    data.to_csv("google_map_result_{}_{}_c.csv".format(prefix,search_string),index = False)
#     print(i, df['zipcode'], t, pd.Timestamp.now() - t)

WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=96.0.4664.110)


In [ ]:
data['ZipCode']=data['Address'].str[-5:].str.strip().astype(np.float).astype("Int32")
data['State']  =data['Address'].str[-8:-6]
data

In [ ]:
county = pd.read_excel(r'C:\Users\Dshahabi\Documents\SampleWorks\HMSA\2019ZipMkt.xlsx')
county["zipcode"]=county["zipcode"].astype(int)
county = county.set_index("zipcode")['County'].to_dict()
data["County"] = data["ZipCode"].map(county)


In [ ]:
data

In [ ]:
data.to_csv("google_map_result_{}_{}_d.csv".format(prefix,search_string),index = False)


In [ ]:
End!